# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

P. Mollière  ->  P. Mollière  |  ['P. Mollière']
E. Matthews  ->  E. Matthews  |  ['E. Matthews']
T. Henning  ->  T. Henning  |  ['T. Henning']
M. Samland  ->  M. Samland  |  ['M. Samland']
S. Scheithauer  ->  S. Scheithauer  |  ['S. Scheithauer']
J. Bouwman  ->  J. Bouwman  |  ['J. Bouwman']
T. Henning  ->  T. Henning  |  ['T. Henning']
J. Shi  ->  J. Shi  |  ['J. Shi']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
J. Shi  ->  J. Shi  |  ['J. Shi']
G. Chauvin  ->  G. Chauvin  |  ['G. Chauvin']


J. Liu  ->  J. Liu  |  ['J. Liu']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
D. Gleis  ->  D. Gleis  |  ['D. Gleis']
A. Hughes  ->  A. Hughes  |  ['A. Hughes']
Arxiv has 69 new papers today
          9 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/9 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2507.18691


extracting tarball to tmp_2507.18691... done.


Found 168 bibliographic references in tmp_2507.18691/main.bbl.
Issues with the citations
syntax error in line 46: '=' expected
Retrieving document from  https://arxiv.org/e-print/2507.18722
extracting tarball to tmp_2507.18722...

 done.


...

extracting tarball to tmp_2507.19342... done.
Retrieving document from  https://arxiv.org/e-print/2507.19439
extracting tarball to tmp_2507.19439...

 done.


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure Figures/Lineintensities_pixelbased_Allsigmas_0.0.png 
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width = 1\textwidth]{Figures/Lineintensities_pixelbased_Allsigmas_0.0.png }
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))


Found 107 bibliographic references in tmp_2507.19439/aa54473-25.bbl.
Error retrieving bib data for colombo_pdbi_2014: 'author'
Retrieving document from  https://arxiv.org/e-print/2507.19480
extracting tarball to tmp_2507.19480...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.18691-b31b1b.svg)](https://arxiv.org/abs/2507.18691) | **Evidence for SiO cloud nucleation in the rogue planet PSO J318**  |
|| <mark>P. Mollière</mark>, et al. -- incl., <mark>E. Matthews</mark>, <mark>T. Henning</mark>, <mark>M. Samland</mark>, <mark>S. Scheithauer</mark> |
|*Appeared on*| *2025-07-28*|
|*Comments*| *30 pages, 9 figures, accepted for publication in A&A*|
|**Abstract**|            Silicate clouds are known to significantly impact the spectra of late L-type brown dwarfs, with observable absorption features at ~ 10 micron. JWST has reopened our window to the mid-infrared with unprecedented sensitivity, bringing the characterization of silicates into focus again. Using JWST, we characterize the planetary-mass brown dwarf PSO J318.5338-22.8603, concentrating on any silicate cloud absorption the object may exhibit. PSO J318's spectrum is extremely red, and its flux is variable, both of which are likely hallmarks of cloud absorption. We present JWST NIRSpec PRISM, G395H, and MIRI MRS observations from 1-18 micron. We introduce a method based on PSO J318's brightness temperature to generate a list of cloud species that are likely present in its atmosphere. We then test for their presence with petitRADTRANS retrievals. Using retrievals and grids from various climate models, we derive bulk parameters from PSO J318's spectra, which are mutually compatible. Our retrieval results point to a solar to slightly super-solar atmospheric C/O, a slightly super-solar metallicity, and a 12C/13C below ISM values. The atmospheric gravity proves difficult to constrain for both retrievals and grid models. Retrievals describing the flux of PSO J318 by mixing two 1-D models (``two-column models'') appear favored over single-column models; this is consistent with PSO J318's variability. The JWST spectra also reveal a pronounced absorption feature at 10 micron. This absorption is best reproduced by introducing a high-altitude cloud layer of small (<0.1 micron), amorphous SiO grains. The retrieved particle size and location of the cloud is consistent with SiO condensing as cloud seeding nuclei. High-altitude clouds comprised of small SiO particles have been suggested in previous studies, therefore the SiO nucleation we potentially observe in PSO J318 could be a more wide-spread phenomenon.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.18722-b31b1b.svg)](https://arxiv.org/abs/2507.18722) | **Detection of CO$_2$, CO, and H$_2$O in the atmosphere of the warm sub-Saturn HAT-P-12b**  |
|| N. Crouzet, et al. -- incl., <mark>J. Bouwman</mark>, <mark>T. Henning</mark> |
|*Appeared on*| *2025-07-28*|
|*Comments*| *22 pages, 16 figures, A&A, accepted*|
|**Abstract**|            The chemical composition of warm gas giant exoplanet atmospheres (with Teq < 1000 K) is not well known due to the lack of observational constraints. HAT-P-12 b is a warm, sub-Saturn-mass transiting exoplanet that is ideal for transmission spectroscopy. One transit of HAT-P-12 b was observed with JWST NIRSpec in the 2.87-5.10 $\mu$m range with a resolving power of $\sim$1000. The JWST data are combined with archival observations from HST WFC3 covering the 1.1-1.7 $\mu$m range. The data are analysed using two data reduction pipelines and two atmospheric retrieval tools. Atmospheric simulations using chemical forward models are performed. CO2, CO, and H2O are detected at 12.2, 4.1, and 6.0 $\sigma$ confidence, respectively. Their volume mixing ratios are consistent with an atmosphere of $\sim$10 times solar metallicity and production of CO2 by photochemistry. CH4 is not detected and seems to be lacking, which could be due to a high intrinsic temperature with strong vertical mixing or other phenomena. SO2 is also not detected and its production seems limited by low upper atmospheric temperatures ($\sim$500 K at $P<10^{-3}$ bar inferred from 1D retrievals), insufficient to produce it in detectable quantities ($\gtrsim$ 800 K required according to photochemical models). Retrievals indicate the presence of clouds between $10^{-1}$ and $10^{-3}$ bar. This study points towards an atmosphere for HAT-P-12 b that could be enriched in carbon and oxygen with respect to its host star. When including the production of CO2 via photochemistry, an atmospheric metallicity that is close to Saturn's can explain the observations. Metallicities inferred for other gas giant exoplanets based on their CO2 mixing ratios may need to account for its photochemical production pathways. This may impact studies on mass-metallicity trends and links between exoplanet atmospheres, interiors, and formation history.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.19439-b31b1b.svg)](https://arxiv.org/abs/2507.19439) | **The SWAN view of dense gas in the Whirlpool -- A cloud-scale comparison of N2H+, HCO+, HNC and HCN emission in M51**  |
|| S. K. Stuber, et al. -- incl., <mark>E. Schinnerer</mark>, <mark>D. Gleis</mark> |
|*Appeared on*| *2025-07-28*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**|            Tracing dense molecular gas, the fuel for star formation, is essential for the understanding of the evolution of molecular clouds and star formation processes. We compare the emission of HCN(1-0), HNC(1-0) and HCO+(1-0) with the emission of N2H+(1-0) at cloud-scales (125 pc) across the central 5x7 kpc of the Whirlpool galaxy, M51a, from "Surveying the Whirlpool galaxy at Arcseconds with NOEMA" (SWAN). We find that the integrated intensities of HCN, HNC and HCO+ are more steeply correlated with N2H+ emission compared to the bulk molecular gas tracer CO, and we find variations in this relation across the center, molecular ring, northern and southern disk of M51. Compared to HCN and HNC emission, the HCO+ emission follows the N2H+ emission more similarly across the environments and physical conditions such as surface densities of molecular gas, stellar mass, star-formation rate, dynamical equilibrium pressure and radius. Under the assumption that N2H+ is a fair tracer of dense gas at these scales, this makes HCO+ a more favorable dense gas tracer than HCN within the inner disk of M51. In all environments within our field of view, even when removing the central 2 kpc, HCN/CO, commonly used to trace average cloud density, is only weakly depending on molecular gas mass surface density. While ratios of other dense gas lines to CO show a steeper dependency on the surface density of molecular gas, it is still shallow in comparison to other nearby star-forming disk galaxies. The reasons might be physical conditions in M51 that are different from other normal star-forming galaxies. Increased ionization rates, increased dynamical equilibrium pressure in the central few kpc and the impact of the dwarf companion galaxy NGC 5195 are proposed mechanisms that might enhance HCN and HNC emission over HCO+ and N2H+ emission at larger-scale environments and cloud scales.         |

## Failed papers

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.18783-b31b1b.svg)](https://arxiv.org/abs/2507.18783) | **SVOM GRB 250314A at z $\simeq$ 7.3: an exploding star in the era of reionization**  |
|| B. Cordier, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-07-28*|
|*Comments*| *12 pages, 11 Figures, 5 Tables, submitted to A&AL*|
|**Abstract**|            Most long Gamma-ray bursts originate from a rare type of massive stellar explosion. Their afterglows, while rapidly fading, can be initially extremely luminous at optical/near-infrared wavelengths, making them detectable at large cosmological distances. Here we report the detection and observations of GRB 250314A by the SVOM satellite and the subsequent follow-up campaign with the near-infrared afterglow discovery and the spectroscopic measurements of its redshift z $\simeq$ 7.3 . This burst happened when the Universe was only $\sim$ 5% of its current age. We discuss the signature of these rare events within the context of the SVOM operating model, and the ways to optimize their identification with adapted ground follow-up observation strategies.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.18785-b31b1b.svg)](https://arxiv.org/abs/2507.18785) | **JWST Observations of SN 2023ixf I: Completing the Early Multi-Wavelength Picture with Plateau-phase Spectroscopy**  |
|| J. M. DerKacy, et al. -- incl., <mark>J. Shi</mark> |
|*Appeared on*| *2025-07-28*|
|*Comments*| *20 pages, 10 figures, submitted to ApJ*|
|**Abstract**|            We present and analyze panchromatic (0.35-14 $\mu$m) spectroscopy of the Type II supernova 2023ixf, including near- and mid-infrared spectra obtained 33.6 days after explosion during the plateau-phase, with the James Webb Space Telescope (JWST). This is the first in a series of papers examining the evolution of SN 2023ixf with JWST spanning the initial 1000 days after explosion, monitoring the formation and growth of molecules and dust in ejecta and surrounding environment. The JWST infrared spectra are overwhelmingly dominated by H lines, whose profiles reveal ejecta structures unseen in optical spectra. We characterize the nature of these structures, concluding that they likely result from a combination of ejecta geometry, viewing angle, and opacity effects. We find no evidence for the formation of dust precursor molecules such as carbon-monoxide (CO), nor do we observe an infrared excess attributable to dust. These observations imply that any future signatures of molecules or dust in SN 2023ixf will arise either from freshly synthesized material within the ejecta or circumstellar material at radii not yet heated by the supernova at this epoch.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.19342-b31b1b.svg)](https://arxiv.org/abs/2507.19342) | **A Truncated Primordial Power Spectrum and its Impact on CMB Polarization**  |
|| <mark>J. Liu</mark>, F. Melia |
|*Appeared on*| *2025-07-28*|
|*Comments*| *19 pages, 24 figures*|
|**Abstract**|            We investigate the impact of a hypothesized delayed initiation of inflation, characterized by a cutoff k_min to the primordial power spectrum in the cosmic microwave background (CMB). This cutoff affects both the scalar and tensor spectra, which therefore impacts several measurements of the temperature and polarization distributions. We calculate the angular power spectrum and correlation function with and without k_min in the context of Planck-LCDM, and demonstrate that a non-zero k_min significantly improves the alignment between theory and the observations, including the temperature, E-mode polarization, TE cross-correlation, Q+U polarization and Q-U polarization. It creates an observable signature in both the angular power spectrum and correlation function for all cases. We thus also explore the B-mode polarization, for which current data are not yet precise enough to determine k_min, but whose impact should be detectable with high-precision measurements using future missions, such as LiteBIRD, if the tensor-to-scalar ratio, r, is not much smaller than its current upper limit. We find that the introduction of k_min not only addresses large-angle anomalies in the CMB but also provides a more consistent framework for understanding the early Universe's inflationary phase. These findings highlight the importance of future high-precision CMB observations in validating the existence and implications of k_min.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.19480-b31b1b.svg)](https://arxiv.org/abs/2507.19480) | **Tracers of the ionization fraction in dense and translucent molecular gas: II. Using mm observations to constrain ionization fraction across Orion B**  |
|| I. Bešlić, et al. -- incl., <mark>A. Hughes</mark> |
|*Appeared on*| *2025-07-28*|
|*Comments*| *Accepted for publication to Astronomy and Astrophysics*|
|**Abstract**|            The ionization fraction ($f_\mathrm{e}=n_\mathrm{e}/n_\mathrm{H}$) is a crucial parameter of interstellar gas, yet estimating it requires deep knowledge of molecular gas chemistry and observations of specific lines, such as those from isotopologs like HCO$^+$ and N$_2$H$^+$, which are detectable only in dense cores. Previous challenges in constraining $f_\mathrm{e}$ over large areas stemmed from the limitations of observational tracers and chemical models. Recent models have identified molecular line ratios that can trace $f_\mathrm{e}$ in different environments within molecular clouds. In this study, we analyze various molecular lines in the 3-4 mm range to derive the ionization fraction across the Orion B giant molecular cloud. We focus on dense and translucent gas, exploring variations with gas density ($n$) and the far-ultraviolet (FUV) radiation field ($G_0$). Our findings show that the ionization fraction ranges from $10^{-5.5}$ to $10^{-4}$ in translucent gas and $10^{-8}$ to $10^{-6}$ in dense gas. Notably, $f_\mathrm{e}$ is sensitive to $G_0$ in dense, UV-illuminated regions, decreasing with increasing volume density ($f_\mathrm{e} \propto n^{-0.227}$ for dense and $f_\mathrm{e} \propto n^{-0.3}$ for translucent gas) and increasing with $G_0$. In translucent gas, differing line ratios yield consistent fe values, indicating the importance of electron excitation of HCN and HNC. For dense gas, we recommend using the CN(1-0)/N$_2$H$^+$(1-0) ratio for upper limits on fe and C$^{18}$O(1-0)/HCO$^+$(1-0) for lower limits. In translucent environments, CCH(1-0)/HNC(1-0) effectively traces $f_\mathrm{e}$. The higher fe values in translucent gas align with the C$^+$/CI/CO transition, while values in dense gas are adequate for coupling with the magnetic field.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.18861-b31b1b.svg)](https://arxiv.org/abs/2507.18861) | **Silicate clouds and a circumplanetary disk in the YSES-1 exoplanet system**  |
|| K. K. W. Hoch, et al. -- incl., <mark>G. Chauvin</mark> |
|*Appeared on*| *2025-07-28*|
|*Comments*| *3 tables, 10 figures, 31 pages, Nature, Vol 643, pages 938-942, 24 July 2025*|
|**Abstract**|            Young exoplanets provide a critical link between understanding planet formation and atmospheric evolution. Direct imaging spectroscopy allows us to infer the properties of young, wide orbit, giant planets with high signal-to-noise. This allows us to compare this young population to exoplanets characterized with transmission spectroscopy, which has indirectly revealed the presence of clouds, photochemistry, and a diversity of atmospheric compositions. Direct detections have also been made for brown dwarfs, but direct studies of young giant planets in the mid-infrared were not possible prior to JWST. With two exoplanets around a solar type star, the YSES-1 system is an ideal laboratory for studying this early phase of exoplanet evolution. We report the first direct observations of silicate clouds in the atmosphere of the exoplanet YSES-1 c through its 9-11 micron absorption feature, and the first circumplanetary disk silicate emission around its sibling planet, YSES-1 b. The clouds of YSES-1 c are composed of either amorphous iron-enriched pyroxene or a combination of amorphous MgSiO3 and Mg2SiO4, with particle sizes of less than or equal to 0.1 micron at 1 millibar of pressure. We attribute the emission from the disk around YSES-1 b to be from submicron olivine dust grains, which may have formed through collisions of planet-forming bodies in the disk.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2507.18691/./all_retrieval_spectra.png', 'tmp_2507.18691/./data_figure.png', 'tmp_2507.18691/./cloud_shape_fits_appendix_corrected_only_full_1.png']
copying  tmp_2507.18691/./all_retrieval_spectra.png to _build/html/
copying  tmp_2507.18691/./data_figure.png to _build/html/
copying  tmp_2507.18691/./cloud_shape_fits_appendix_corrected_only_full_1.png to _build/html/
exported in  _build/html/2507.18691.md
    + _build/html/tmp_2507.18691/./all_retrieval_spectra.png
    + _build/html/tmp_2507.18691/./data_figure.png
    + _build/html/tmp_2507.18691/./cloud_shape_fits_appendix_corrected_only_full_1.png
found figures ['tmp_2507.18722/./figures/HAT-P-12b_transmission_spectra.png', 'tmp_2507.18722/./figures/comp_prob_dis.png', 'tmp_2507.18722/./figures/HAT-P-12b-chemistryresults.png']
copying  tmp_2507.18722/./figures/HAT-P-12b_transmission_spectra.png to _build/html/
copying  tmp_2507.18722/./figures/comp_prob_dis.png to _build/html/
copying  tmp_2507.18722/./figures/HAT

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\rch}[1]{{{\color{black}#1}}}$
$\newcommand{\todoSO}[1]{\stepcounter{todocounter}\oldtodo[inline, color=green]{\thetodocounter: Paul -- #1}}$
$\newcommand{\todo}[1]{\stepcounter{todocounter}\oldtodo[inline]{\thetodocounter: #1}}$
$\newcommand{\mjup}{{\rm M_{\jupiter}}}$
$\newcommand{\micron}{µm}$
$\newcommand{\pso}{PSO~J318}$
$\newcommand{\aj}{AJ}$
$\newcommand{\araa}{ARA\&A}$
$\newcommand{\apj}{ApJ}$
$\newcommand{\apjl}{ApJ}$
$\newcommand{\apjs}{ApJS}$
$\newcommand{\ao}{Appl.Optics}$
$\newcommand{\apss}{Ap\&SS}$
$\newcommand{\aap}{A\&A}$
$\newcommand{\aapr}{A\&A~Rev.}$
$\newcommand{\aaps}{A\&AS}$
$\newcommand{\baas}{BAAS}$
$\newcommand{\jrasc}{JRASC}$
$\newcommand{\memras}{MmRAS}$
$\newcommand{\mnras}{MNRAS}$
$\newcommand{\pra}{Phys.Rev.A}$
$\newcommand{\prb}{Phys.Rev.B}$
$\newcommand{\prc}{Phys.Rev.C}$
$\newcommand{\prd}{Phys.Rev.D}$
$\newcommand{\prl}{Phys.Rev.Lett}$
$\newcommand{\pasp}{PASP}$
$\newcommand{\pasj}{PASJ}$
$\newcommand{\qjras}{QJRAS}$
$\newcommand{\skytel}{S\&T}$
$\newcommand{\solphys}{Solar~Phys.}$
$\newcommand{\sovast}{Soviet~Ast.}$
$\newcommand{\ssr}{Space~Sci.Rev.}$
$\newcommand{\zap}{ZAp}$
$\newcommand{\ActaA}{{Acta Astron.}}$</div>



<div id="title">

# Evidence for SiO cloud  nucleation in the rogue planet PSO J318

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.18691-b31b1b.svg)](https://arxiv.org/abs/2507.18691)<mark>Appeared on: 2025-07-28</mark> -  _30 pages, 9 figures, accepted for publication in A&A_

</div>
<div id="authors">

<mark>P. Mollière</mark>, et al. -- incl., <mark>E. Matthews</mark>, <mark>T. Henning</mark>, <mark>M. Samland</mark>, <mark>S. Scheithauer</mark>

</div>
<div id="abstract">

**Abstract:** Silicate clouds have long been known to significantly impact the spectra of late L-type brown dwarfs, with observable absorption features at $\sim 10$ µm. The $*James Webb Space Telescope*$ ( $*JWST*$ ) has reopened our window to the mid-infrared with unprecedented sensitivity, bringing the characterization of silicate clouds into focus again. Using $*JWST*$ , we aim to characterize the planetary-mass brown dwarf PSO J318.5338-22.8603, concentrating on any silicate cloud absorption the object may exhibit. $\pso$ 's spectrum is extremely red, and its flux is variable, both of which are thought to be hallmarks of cloud absorption. We present $*JWST*$ $*NIRSpec PRISM*$ , $*G395H*$ , and $*MIRI MRS*$ observations of $\pso$ from 1 to 18 µm. We introduce a method based on $\pso$ 's brightness temperature to generate a list of cloud species that are likely present in its atmosphere. We then test for the species' presence with \texttt{petitRADTRANS} retrievals. Using retrievals and grids from various climate models, we derive bulk parameters from $\pso$ 's spectra, which are mutually compatible. Our retrieval results point to a solar to slightly super-solar atmospheric C/O, a slightly super-solar metallicity, and a $\rm ^{12}C/^{13}C$ below ISM values. The atmospheric gravity proves difficult to constrain for both retrievals and grid models. Retrievals describing the flux of $\pso$ by mixing two 1-D models ("two-column models") appear favored over single-column models; this is consistent with $\pso$ 's variability. The $*JWST*$ observations also reveal a pronounced absorption feature at $10$ µm. This absorption is best reproduced by introducing a high-altitude cloud layer of small ( $<$ 0.1 µm), amorphous $\ce{SiO}$ grains. The retrieved particle size and location of the cloud is consistent with $\rch{SiO condensing as cloud seeding}$ nuclei. High-altitude clouds comprised of small SiO particles have been suggested in previous studies, therefore the SiO nucleation we potentially observe in $\pso$ could be a more wide-spread phenomenon.

</div>

<div id="div_fig1">

<img src="tmp_2507.18691/./all_retrieval_spectra.png" alt="Fig8" width="100%"/>

**Figure 8. -** Best-fit spectra for all retrieval models listed in Table \ref{tab:all_retrieval_posteriors}. From left to right, top to bottom, the ordering is identical to the model ordering of Table \ref{tab:all_retrieval_posteriors}. The residuals of the winning model (SiO cloud with amorphous spherical particles in a two-column model with a global Fe cloud) are plotted in black below the respective residuals of all other models. (*fig:best_fits_all_retrievals*)

</div>
<div id="div_fig2">

<img src="tmp_2507.18691/./data_figure.png" alt="Fig2" width="100%"/>

**Figure 2. -** $\rch${Observations considered for the atmospheric characterization of $\pso$.}_ Uppermost panel:_ all data considered for the spectral characterization of $\pso$, at the wavelength binning fed into the retrieval and self-consistent grid model fits. Specifically, *MIRI MRS* data was binned down to $\lambda/\Delta\lambda=400$, while NIRSpec G395H was binned to $\lambda/\Delta\lambda=400$ and $\lambda/\Delta\lambda=1000$ for NRS1 and NRS2, respectively. _ Second panel:_ the *HST WFC3*, *IRTF SpeX* and *JWST NIRSpec PRISM* data over the 1-3 µm wavelength range. _ Third panel:_ the *JWST NIRSpec G395H* data at the full spectral resolution. _ Lowermost panel:_ the *JWST MIRI MRS* data at the full spectral resolution. (*fig:jwstdata*)

</div>
<div id="div_fig3">

<img src="tmp_2507.18691/./cloud_shape_fits_appendix_corrected_only_full_1.png" alt="Fig3" width="100%"/>

**Figure 3. -** $\rch${Opacity fits of the $10$ µm feature for the "Top-14" silicate cloud species, using the brightness temperature method. The remaining fits for the less well fitting species can be found in Fig. \ref{fig:brightness_temperature_all_appendix_1}} (*fig:cloud_shape_fits*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.18691"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\jwst}{\textit{JWST}\xspace}$
$\newcommand{\hst}{\textit{HST}\xspace}$
$\newcommand{\spitzer}{\textit{Spitzer}\xspace}$
$\newcommand{\kepler}{\textit{Kepler}\xspace}$
$\newcommand{\tess}{\textit{TESS}\xspace}$
$\newcommand{\nirspec}{NIRSpec\xspace}$
$\newcommand{\teatro}{\texttt{TEATRO}\xspace}$
$\newcommand{\cascade}{\texttt{CASCADe}\xspace}$
$\newcommand{\vulcan}{\texttt{VULCAN}\xspace}$
$\newcommand{\arcis}{\texttt{ARCiS}\xspace}$
$\newcommand{\taurex}{\texttt{TauREx}\xspace}$
$\newcommand{\petitradtrans}{\texttt{petitRADTRANS}\xspace}$
$\newcommand{\pyratbay}{\texttt{PYRAT\;BAY}\xspace}$
$\newcommand{\exotethys}{\texttt{ExoTETHyS}\xspace}$
$\newcommand{\mum}{\mum\xspace}$
$\newcommand{\eg}{{e.g.,}\xspace}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.1}$
$\newcommand{\arraystretch}{1.1}$
$\newcommand{\arraystretch}{1}$
$\newcommand{\arraystretch}{1.2}$</div>



<div id="title">

# Detection of $CO_2$, CO, and $H_2$O in the atmosphere of the warm sub-Saturn HAT-P-12 b

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.18722-b31b1b.svg)](https://arxiv.org/abs/2507.18722)<mark>Appeared on: 2025-07-28</mark> -  _22 pages, 16 figures, A&A, accepted_

</div>
<div id="authors">

N. Crouzet, et al. -- incl., <mark>J. Bouwman</mark>, <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** The chemical composition of warm gas giant exoplanet atmospheres (with $T_{\rm eq} < 1000 \;\rm K$ ) is not well known due to the lack of observational constraints. A new era began for the study of exoplanet atmospheres with the $\jwst$ . HAT-P-12 b is a warm, sub-Saturn-mass transiting exoplanet that is ideal for transmission spectroscopy. We aim to characterise its atmosphere and probe the presence of carbonaceous species using near-infrared observations. One transit of HAT-P-12 b was observed in spectroscopy with $\jwst$ NIRSpec in the $2.87-5.10 \; \mu$ m range with a resolving power of $\sim$ 1000. The $\jwst$ data are combined with archival observations from $\hst$ WFC3 covering the $1.1-1.7 \; \mu$ m range. The data are analysed using two data reduction pipelines and two atmospheric retrieval tools. Atmospheric simulations using chemical forward models are performed to interpret the spectra. $CO_2$ , CO, and $H_2$ O are detected at 12.2, 4.1, and 6.0 $\sigma$ confidence, respectively. Their volume mixing ratios are consistent with an atmosphere of $\sim$ 10 times solar metallicity and production of $\ce{CO2}$ by photochemistry. $CH_4$ is not detected and seems to be lacking, which could be due to a high intrinsic temperature with strong vertical mixing or other phenomena. $SO_2$ is also not detected and its production seems limited by low upper atmospheric temperatures ( $\sim$ 500 K at $P<10^{-3}$ bar inferred from 1D retrievals), insufficient to produce it in detectable quantities ( $\gtrsim$ 800 K required according to photochemical models). $H_2$ S is marginally detected using one data analysis method and not by the other. Retrievals indicate the presence of clouds between $10^{-1}$ and $10^{-3}$ bar. The inferred C/O ratio is below unity but is not well constrained. This study points towards an atmosphere for HAT-P-12 b that could be enriched in carbon and oxygen with respect to its host star, a possibly cold upper atmosphere that is unfavourable to $\ce{SO2}$ production and may explain its non-detection, and a $\ce{CH4}$ depletion that is yet to be fully understood. When including the production of $\ce{CO2}$ via photochemistry, an atmospheric metallicity that is close to Saturn's can explain the observations. Metallicities inferred for other gas giant exoplanets based on their $\ce{CO2}$ mixing ratios may need to account for its photochemical production pathways. This may impact studies on mass-metallicity trends and links between exoplanet atmospheres, interiors, and formation history.

</div>

<div id="div_fig1">

<img src="tmp_2507.18722/./figures/HAT-P-12b_transmission_spectra.png" alt="Fig7" width="100%"/>

**Figure 7. -** Synthetic spectra with grey clouds based on $\vulcan$ chemistry models for \mbox{HAT-P-12 b}. From top to bottom, the metallicity increases from 1 to 60 times solar. Models with carbon-to-oxygen ratios of 0.25 (orange), 0.50 (black), and 0.75 (green) with (solid lines) and without (dotted lines) photochemistry are shown. An offset is applied to the reference radius-pressure points of the model spectra to match the observed NIRSpec transit depths, and an offset of $0.09 \%$ is added for visual purposes. The $\jwst$ NIRSpec G395M data ($\teatro$ reduction) and the $\hst$ WFC3 G141 data are shown in light grey. The coloured regions correspond to the contribution of individual species to the model spectrum with C/O of 0.50 and photochemistry, which is also superimposed on the data. (*fig:vulcan_spectra*)

</div>
<div id="div_fig2">

<img src="tmp_2507.18722/./figures/comp_prob_dis.png" alt="Fig11" width="100%"/>

**Figure 11. -** Probability distributions for different molecules for each retrieval conducted here. One sigma constraints are also shown and arrows indicate 1$\sigma$ upper bounds on the presence of a species. (*fig:abundances*)

</div>
<div id="div_fig3">

<img src="tmp_2507.18722/./figures/HAT-P-12b-chemistryresults.png" alt="Fig12" width="100%"/>

**Figure 12. -** $\vulcan$ chemistry results for HAT-P-12 b. The upper left panel shows the $P-T$ profile with high intrinsic temperature, resulting from the optimisation procedure described in Sect. \ref{subsec: forward models methods}. The other panels show the VMR of different molecules detected or expected in the NIRSpec spectrum, for different metallicities, carbon-to-oxygen ratios, with and without photochemistry. From dark to light grey, the shaded areas indicate respectively the 1, 2, and $3 \sigma$ uncertainty intervals of the retrieved atmospheric temperature and of the VMRs for detected species, and the 1, 2, and $3 \sigma$ upper limits of the VMRs for non-detected species (inferred from the $\hst$ + NIRSpec $\teatro$ spectrum and $\arcis$ retrieval). (*fig:vulcan*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.18722"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\nnhp}[0]{N_2H^+\xspace}$
$\newcommand{\hcop}[0]{HCO^+\xspace}$
$\newcommand{\tco}[0]{^{13}CO\xspace}$
$\newcommand{\CO}[0]{^{12}CO\xspace}$
$\newcommand{\ncrit}[0]{n_\mathrm{crit}\xspace}$
$\newcommand{\neff}[0]{n_\mathrm{eff}\xspace}$
$\newcommand{\SSFR}[0]{\Sigma_\mathrm{SFR}\xspace}$
$\newcommand{\Sstar}[0]{\Sigma_\ast\xspace}$
$\newcommand{\Smol}[0]{\Sigma_\mathrm{mol}\xspace}$
$\newcommand{\Pde}[0]{P_\mathrm{DE}\xspace}$
$\newcommand{\vdisp}[0]{\sigma_\mathrm{CO}\xspace}$
$\newcommand{\dep}[0]{\tau_\mathrm{depl}\xspace}$
$\newcommand{\refs}[0]{\textbf{\textcolor{red}{ref}}\xspace}$</div>



<div id="title">

# The SWAN view of dense gas in the Whirlpool: A cloud-scale comparison of $\nnhp$, $\hcop$, HNC and HCN emission in M51

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.19439-b31b1b.svg)](https://arxiv.org/abs/2507.19439)<mark>Appeared on: 2025-07-28</mark> -  _Accepted for publication in A&A_

</div>
<div id="authors">

S. K. Stuber, et al. -- incl., <mark>E. Schinnerer</mark>, <mark>D. Gleis</mark>

</div>
<div id="abstract">

**Abstract:** Tracing dense molecular gas, the fuel for star formation, is essential for the understanding of the evolution of molecular clouds and star formation processes.  We compare the emission of HCN (1-0), HNC (1-0) and $\hcop$ (1-0) with the emission of $\nnhp$ (1-0) at cloud-scales ( $125 $ pc) across the central $5\times7 $ kpc of the Whirlpool galaxy, M51a, from "Surveying the Whirlpool galaxy at Arcseconds with NOEMA" (SWAN).  We find that the integrated intensities of HCN, HNC and $\hcop$ are more steeply correlated with $\nnhp$ emission compared to the bulk molecular gas tracer CO, and we find variations in this relation across the center, molecular ring, northern and southern disk of M51.  Compared to HCN and HNC emission, the $\hcop$ emission follows the $\nnhp$ emission more similarly across the environments and physical conditions such as surface densities of molecular gas, stellar mass, star-formation rate, dynamical equilibrium pressure and radius.  Under the assumption that $\nnhp$ is a fair tracer of dense gas at these scales, this makes $\hcop$ a more favorable dense gas tracer than HCN within the inner disk of M51.  In all environments within our field of view, even when removing the central 2 kpc, HCN/CO, commonly used to trace average cloud density, is only weakly depending on molecular gas mass surface density.  While ratios of other dense gas lines to CO show a steeper dependency on the surface density of molecular gas, it is still shallow in comparison to other nearby star-forming disk galaxies.  The reasons might be  physical conditions in M51 that are different from other normal star-forming galaxies.  Increased ionization rates, increased dynamical equilibrium pressure in the central few kpc and the impact of the dwarf companion galaxy NGC 5195 are proposed mechanisms that might enhance HCN and HNC emission over $\hcop$ and $\nnhp$ emission at larger-scale environments and cloud scales.

</div>

<div id="div_fig1">

<img src="tmp_2507.19439/./Figures/Gallery_w_hist.png" alt="Fig17" width="100%"/>

**Figure 17. -** Integrated intensity maps of dense gas tracers HCN, HNC, HCO+ and $\nnhp$ from SWAN at a common resolution of $3$\arcsec$$(top row from left to right), as well as from $\CO$(PAWS, bottom left). We divide the disk into a center and ring environment  (colombo_pdbi_2014) , and the outer disk into northern and southern halves (bottom row, second panel from left). We add contours of integrated $\nnhp$ emission of 0.75, 2 and 4 K km/s to the environment map.
 We show the pixel-based integrated intensity distribution (in K km/s) in various environments in the disk for all pixels in the FoV (colored shaded area), as well as for pixels where emission is detected (emission $>3\sigma$, light grey shaded area) in the bottom right panels. The area of each histogram is normalized to unity.
 We indicate the median of all pixels (black dashed grey line) and median of masked pixels (dotted grey line) of each environment.
 The median represents the median value of logarithmic emission (med$\left(\mathrm{log}_{10}\left( I \right) \right)$. Pixels with negative emission are excluded in the logarithmic scaling of the histograms.
 Since CO is detected across most of the FoV and is used as a prior in the creation of the moment-0 map (Section \ref{sec:Data:SWANmoments}), its masked histogram distribution agrees well with the unmasked one.
  (*fig:Gallery*)

</div>
<div id="div_fig2">

<img src="tmp_2507.19439/./Figures/Gallery_Lineratios_w_Histograms_divnorm_histomedian_bwr_n2hpcontours.png" alt="Fig19" width="100%"/>

**Figure 19. -** Left panel: Line ratios of integrated line emission from dense gas tracers HCN, HNC and $\hcop$. For visual purpose, we only show line ratios for significant detected pixels ($>3\sigma$), but include non-detections in all calculations. We mark pixels in which CO is detected (grey points) and the center of the galaxy (green plus). The intensity scale is centered logarithmically on the average line ratios (log$_{10} R$, with $R$ from Table \ref{tab:Averagelineratios}) determined for all pixels in the FoV, including non-detections.  The average line ratio (log$_{10} R$) and the total range of 1 dex covered by the color bar are indicated by the black dashed line and greyshaded area in the right panel. We show contours of integrated $\nnhp$ emission (0.75, 3 K km/s) on top.
    Right panel: Histogram of line ratios per environment analogous to Figure \ref{fig:Gallery} but for HCN/HNC, HCN/$\hcop$ and HNC/$\hcop$(colored histograms).
    We indicate the amount of pixels shown in the histogram (top right corner), which varies slightly, as values with negative noise can not be shown in the logarithmic scale.
     (*fig:Gallery_linerationon2hp*)

</div>
<div id="div_fig3">

<img src="tmp_2507.19439/./Figures/Intensitiesaverage_Alllines_noAGN_0.0_Allsigmas.png" alt="Fig2" width="100%"/>

**Figure 2. -** Average line intensity $I_\mathrm{Line}$ as a function of surface densities of molecular gas mass ($\Smol$), stellar mass ($\Sstar$) and star formation rate ($\SSFR$), as well as velocity dispersion ($\vdisp$), dynamical equilibrium pressure ($\Pde$) and galactocentric radius for all dense gas tracers.
    Shaded areas mark the standard deviation per bin. The $\CO$ intensity scaled by a factor of $1/10$ is added for comparison (grey line).
    For $\rm r>2.2kpc$, the shape of our FoV leads to incomplete sampling of these radial bins (grey shaded area). The obtained slopes for a linear fit (in log space) to the binned averages are provided in each panel. (*fig:average_intensity_sigmas*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.19439"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

152  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

13  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
